In [ ]:
# umbridge server
from scipy.integrate import solve_ivp # for solving the ode
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from mpl_toolkits.mplot3d import axes3d
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from scipy.linalg import block_diag
import umbridge
from scipy import linalg

class PDEModel(umbridge.Model):
    def __init__(self):
        super().__init__("forward")
        self.C= np.diag([2, 0, 0, 0, 2, 2])
        self.T = np.ones((6, 6))
        self.T[0] = 1/np.sqrt(6) * np.ones((1,6))
        self.T[1, 0:3] = - 1/np.sqrt(6) * np.ones((1,3))
        self.T[1, 3:6] = 1/np.sqrt(6) * np.ones((1,3))
        self.T[2] = [1/2, -1/2, 0, -1/2, 1/2, 0]
        self.T[3] = [1/ np.sqrt(12), 1/ np.sqrt(12), -1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
        self.T[4] = [-1/2, 1/2, 0, -1/2, 1/2, 0]
        self.T[5] = [-1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
        self.n = 6
        self.external_noise = [0] * 4  # Initialize with the required size (4 elements)
        self.U_AC = 325
        self.U_DC = 700

        
        
        
    def get_input_sizes(self, config):
        return [1, 1, 1, 1, 1, 1, 6, 6, 1, 1] 
 # for the values 6 is for R_arm, L_arm etc., 6 is for u_1:6, initial conditions for x, angular frequency, time  
    
    def get_output_sizes(self, config):
        return [6]  # for the output variables x_1:6
        
    def __call__(self, parameters, config):
        R_arm = parameters [0][0]
        R_DC = parameters [1][0]
        R_AC = parameters [2][0]
        L_arm = parameters [3][0]
        L_DC = parameters [4][0]
        L_AC = parameters [5][0]
        u_parameter = parameters [6]
        x_initial = parameters [7]
        omega = parameters[8][0]
        T = parameters[9][0]
        matrix_A = self.a(self.n, R_arm, R_DC, L_arm, L_DC, R_AC, L_AC, config)
        matrix_B = self.b(self.n, L_arm, L_DC, L_AC, config)
        matrix_F = self.f(self.n, L_arm, L_DC, L_AC, config)
        #z_vector = self.z(T, omega)
        sol = solve_ivp(self.MMC, [0, T], x_initial, args=(matrix_A, matrix_B, matrix_F, u_parameter, config), method='RK45', t_eval=[T])
        result = [sol.y[0][0], sol.y[1][0], sol.y[2][0], sol.y[3][0], sol.y[4][0], sol.y[5][0]]
        return [result]



    # A matrix
    def a(self, n, R_arm, R_DC, L_arm, L_DC, R_AC, L_AC, config):
        A = np.zeros((self.n, self.n))
        np.fill_diagonal(A, [0, -(R_arm+3*R_DC)/(L_arm+3*L_DC), -(R_arm)/(L_arm), -(R_arm)/(L_arm), -(R_arm+2*R_AC)/(L_arm+2*L_AC), -(R_arm+2*R_AC)/(L_arm+2*L_AC)])
        return A  

    # B matrix
    def b(self, n, L_arm, L_DC, L_AC, config):
        B = np.zeros((self.n, self.n))
        np.fill_diagonal(B, [0, -(1)/(L_arm+3*L_DC), -(1)/(L_arm), -(1)/(L_arm), -(1)/(L_arm+2*L_AC), -(1)/(L_arm+2*L_AC)])
        return B  


    # F matrix
    def f(self, n, L_arm, L_DC, L_AC, config):
        F = np.zeros((self.n , self.n -2))
        F[1][3] = (-(np.sqrt(6)))/(2*L_arm+6*L_DC)
        F[4][0] = (1/(L_arm+2*L_AC))
        F[4][1] = (-1/(L_arm+2*L_AC))
        F[5][0] = (np.sqrt(3)/(3*L_arm+6*L_AC))
        F[5][1] = (np.sqrt(3)/(3*L_arm+6*L_AC))
        F[5][2] = (-(np.sqrt(3))/(3*L_arm+6*L_AC))
        return F  

    def z(self, T, omega, config):
        
        for k in range(1, 4):
            self.external_noise[k-1]= self.U_AC * np.cos (omega * T - (2 * np.pi * (k-1))/3)

        self.external_noise[3] = self.U_DC
        return self.external_noise
            

    # system of equations
    def MMC(self, T, x, A, B, F, u, config):
        
        dxdt = A @ x + B @ u + F @ self.external_noise

        return dxdt

   
    def supports_evaluate(self):
        return True

pde_model = PDEModel()
umbridge.serve_models([pde_model], 4242) # start model server

======== Running on http://0.0.0.0:4242 ========
(Press CTRL+C to quit)
